# Advanced Certification Program in Computational Data Science

##  A program by IISc and TalentSprint

### Mini Project Notebook 1 : Data analytics



## Learning Objectives



At the end of the experiment, you will be able to :


* understand the requirements for a “clean” dataset, ready for use in statistical analysis

* use Python libraries like Pandas, Numpy, and Matplotlib to perform the  data-preprocessing steps

* obtain probability and statistics based insights from the data


## Dataset

The dataset chosen for this experiment is the **Play Store** dataset which is  publicly available and created with this [methodology](https://nycdatascience.com/blog/student-works/google-play-store-everything-that-you-need-to-know-about-the-android-market/)  

This dataset consists of 10841 records. Each record is made up of 13 fields.

**For example**, Each record consists of App, Category, Rating, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver, and Android Ver.

## Problem Statement

Before we can derive any meaningful insights from the Play Store data, it is essential to pre-process the data and make it suitable for further analysis. This pre-processing step forms a major part of data wrangling (or data munging) and ensures better quality data. It consists of the transformation and mapping of data from a "raw" data form into another format so that it is more valuable for a variety of downstream purposes such as analytics. Data analysts typically spend a sizeable amount of time in the process of data wrangling, compared to the actual analysis of the data.

After data munging is performed, several actionable insights can be derived from the Play Store apps data. Such insights could help to unlock the enormous potential to drive app-making businesses to success.

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/googleplaystore.csv

#### Import required packages

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

#### Load the dataset

In [ ]:
# YOUR CODE HERE
df = pd.read_csv("googleplaystore.csv")
df.info()
df.head()

## Pre-processing

In [ ]:
df = pd.read_csv("googleplaystore.csv")
# YOUR CODE HERE
import math

print(df.isnull().sum())
# Decide what values make sense for the columns where there are null
df = df.fillna({'Rating': 0.0}), 'Type': 'Free', 'Current Ver': 'Not Available', 'Android Ver': 'Not Available', 'Content Rating': 'Not rated'})

print(df.isnull().sum())

# converting reviews into numericals
df['Reviews'] = df['Reviews'].apply(
    lambda d: int(float(d.upper().replace('M', ''))*1000000) if 'M' in d.upper() else int(float(d.upper().replace('K', ''))*1000) if 'K' in d.upper() else int(d) )

# removing duplicates
df_dupl = df[df.duplicated(subset=['App'])]
print('\n Before: Duplicate entries for apps is : {} out of {}'.format(df_dupl.shape[0], df.shape[0]))
# there could be multiple entries for one app - which entry should be kept in that case?
# we can keep the one that has max values - assuming that might be latest
df = df.sort_values(['Last Updated', 'Rating', 'Reviews', 'Installs'], ascending=False).drop_duplicates(['App']).sort_index(ignore_index=True)

# remove apps with non english characters
df_non_englsih = df[df['App'].apply(lambda d: not(str(d).isascii()))]
print('\n Apps with Non English chars: {} out of {}'.format(df_non_englsih.shape[0], df.shape[0]))
df = df[df['App'].apply(lambda d: str(d).isascii())]
print('\n Apps after removing non english : ', df.shape[0])

df = df[df['Category'].apply(lambda d: str(d).isascii() and str(d).upper().isupper())]
print('\n Apps after removing invalid categories : ', df.shape[0])

# # getting average of the rating per category
# df_cat_rating = df.groupby('Category')['Rating'].aggregate('mean').drop_duplicates()
# df['Rating'] = df.apply(lambda r: df_cat_rating[r['Category']] if math.isnan(r['Rating']) else r['Rating'], axis=1)
# df = df[df['Rating'] <= 5.0]
# df = df['Rating'].fillna(0.0)

print('\n Final: Number of apps is {}'.format(df.shape[0]))

# converting size to float, in MBs
translation = {'+': '', '+': '', 'k': '000', 'K': '000', 'M': '000000', 'm': '000000', ',':'', '.': ''}
transtable = ''.maketrans(translation)
df['Size'] = df['Size'].apply(lambda d: float(d.upper().translate(transtable))/1000000 if '0' <= d[0] <= '9' else 0)
# lambda d: float(d.upper().replace('M', ''))*1000000 if 'M' in d.upper() else float(d.upper().replace('K', ''))*1000 if 'K' in d.upper() else float(str(d).replace('+', '')) if '0' <= d[0] <= '9' else 0)

# Converting price to float and assuming all in dollars
df['Price'] = df['Price'].apply(lambda d: float(d.replace('$', '')) if d[0] == '$' else 0)
# Converting installs into number, removing '+'
df['Installs'] = df['Installs'].apply(lambda d: int(str(d).replace('+', '').replace(',', '')) if '0' <= d[0] <= '9' else 0)

# print(df['Installs'].unique())
# print(df['Size'].unique())
# print(df['Category'].unique())

### Task 1: Data Cleaning

* Check whether there are any null values and figure out how you want to handle them?
  
    **Hint:** isnan(), dropna(), fillna()
* If there is any duplication of a record, how would you like to handle it?

    Hint: [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html)

* Are there any non-English apps? And how to filter them?

* In the size column, multiply 1,000,000 with M in the cell and multiply by 1000 if we have K in the cell.

## Visualization

### Task 2: Perform the  following tasks:

##### Exercise 1: Find the number of apps in various categories by using an appropriate plot.

In [ ]:
# YOUR CODE HERE
import seaborn.objects as so

app_count = df['Category'].value_counts()
sns.set(style="darkgrid")
sns.barplot(x=app_count.index, y=app_count.values, alpha=0.5, palette='turbo')

# so.Plot(x=app_count.index, y=app_count.values).add(so.Bar(), so.Dodge(gap=0.4))
plt.title('Number of Apps in Various Categories')
plt.ylabel('Number of Apps', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (12,6)
plt.show()

##### Exercise 2: Explore the distribution of free and paid apps across different categories

**Hint:** Stacked Bar Graph, [link](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/bar_stacked.html)

In [ ]:
# YOUR CODE HERE

categories_count = df[df['Type'].isin(['Paid','Free'])]['Category'].value_counts()
app_free_count = df[df['Type']=='Free']['Category'].value_counts()
app_paid_count = df[df['Type']=='Paid']['Category'].value_counts()

data = pd.DataFrame()
data['Category'] = categories_count.index
data = data.set_index('Category')
data.loc[categories_count.index, 'Free'] = app_free_count
data.loc[categories_count.index, 'Paid'] = app_paid_count

p1 = plt.bar(data.index, data['Free'], width=0.8)
p2 = plt.bar(data.index, data['Paid'], width=0.8, bottom=data['Free'])

plt.title('Number of Apps in Various Categories')
plt.ylabel('Number of Apps', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)

plt.legend((p1[0], p2[0]), ('Free', 'Paid'))
plt.rcParams["figure.figsize"] = (12,6)
plt.show()

# data.plot(kind='bar', stacked=True)

##### Exercise 3: Represent the distribution of app rating on a scale of 1-5 using an appropriate plot

**Hint:** histogram / strip plot

In [ ]:
# YOUR CODE HERE
# using the seaborn histogram
sns.histplot(data=df, x="Rating", bins=20, binwidth=0.25, binrange=(0.0,5.0), stat='count')

plt.title('App rating distribution')
plt.ylabel('Number of Apps', fontsize=12)
plt.xlabel('Rating', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)

plt.rcParams["figure.figsize"] = (12,6)
plt.show()

# Plotting a basic histogram sing matplotlib
# plt.hist(df['Rating'], color='skyblue', edgecolor='black', bins=24, range=(0,6.0))


##### Exercise 4: Identify outliers of the rating column by plotting the boxplot category wise and handle them.

**Hint:** Removing outliers using Z-score, quantile [link](https://kanoki.org/2020/04/23/how-to-remove-outliers-in-python/)

In [ ]:
# YOUR CODE HERE
from scipy import stats

# df.boxplot('Rating', 'Category', rot=90, figsize=(15,8))
sns.boxplot(data=df, x="Category", y="Rating", width=0.5)
plt.ylim(0, 6)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (12,6)



In [ ]:
# calculate the Z scores
for g,d in df.groupby('Category'):
    df.loc[df['Category']== g, 'z_score'] = stats.zscore(d['Rating'])

# remove outliers where z score is > 3
df_processed = df[np.absolute(df['z_score']) < 3]

sns.boxplot(data=df_processed, x="Category", y="Rating", width=0.5)
plt.ylim(0, 6)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (12,6)

In [ ]:
# calculate the quartile scores
for g,d in df.groupby('Category'):
    q1 = d['Rating'].quantile(0.25)
    q3 = d['Rating'].quantile(0.75)
    df.loc[df['Category']==g, 'Q1'] = q1
    df.loc[df['Category']==g, 'Q3'] = q3
    df.loc[df['Category']==g, 'IQR'] = q3-q1

filter = (df['Rating'] >= df['Q1'] - 1.5 * df['IQR']) & (df['Rating'] <= df['Q3'] + 1.5 * df['IQR'])
df_processed_1 = df.loc[filter]

sns.boxplot(data=df_processed_1, x="Category", y="Rating", width=0.5, showfliers=False)
plt.ylim(0, 6)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (12,6)

##### Exercise 5: Plot the barplot of all the categories indicating no. of installs

In [ ]:
# YOUR CODE HERE
import seaborn.objects as so

install_total = df.groupby('Category')['Installs'].sum()
sns.set(style="darkgrid")
sns.barplot(x=install_total.index, y=install_total.values, alpha=0.5, palette='turbo')

# so.Plot(x=app_count.index, y=app_count.values).add(so.Bar(), so.Dodge(gap=0.4))
plt.title('Number of Installs in Various Categories')
plt.ylabel('Number of Installs', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (12,6)
plt.show()

## Insights


### Task 3: Derive the below insights

##### Exercise 1: Does the price correlate with the size of the app?

  **Hint:** plot the scatterplot of `Size` and `Price`

In [ ]:
# YOUR CODE HERE
sns.set(style='whitegrid')
# sns.scatterplot(x=df['Size'], y=df['Price'])
plt.scatter(df['Size'],df['Price'])
plt.title('Size Vs Price correlation')
plt.ylabel('Price in dollars', fontsize=12)
plt.xlabel('Size in MB', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=10,
               grid_alpha=0.5, axis='y', labelsize='small', labelrotation=0)

plt.rcParams["figure.figsize"] = (30,6)
plt.show()


##### Exercise 2: Find the popular app categories based on rating and no. of installs

**Hint:** [df.groupby.agg()](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html); Taking the average rating could be another approach



In [ ]:
# YOUR CODE HERE
df_top_cats = df.groupby('Category').agg({'Rating': 'mean', 'Installs': 'sum'}).sort_values(['Rating', 'Installs'], ascending=False)
print("The top 10 popular categories are: \n")
print(df_top_cats.head(10))

##### Exercise 3: How many apps are produced in each year category-wise ?

  * Create a `Year` column by slicing the values of `Last Updated` column and find the Year with most no. of apps produced

    **For example**, slice the year `2017` from `February 8, 2017`

  * Find the categories which have a consistent rating in each year

      **Hint:** `sns.countplot`

In [ ]:
# YOUR CODE HERE
df['Year'] = df['Last Updated'].apply(lambda d: int(d.split()[2]))
sns.countplot(x ='Category', hue = "Year", data = df)

plt.title('Number of Apps in Various Categories Year wise')
plt.ylabel('Number of Apps', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (20,6)


In [ ]:
df_high_rated = df[df['Rating']>=3.0]
# count plot on two categorical variable
sns.countplot(x ='Category', data=df_high_rated)

plt.title('Consistent high rated Categories')
plt.ylabel('Number of Apps', fontsize=12)
plt.xlabel('Category', fontsize=12)
plt.margins(0.02)
plt.tick_params(direction='out', length=6, width=2,
               grid_alpha=0.5, axis='x', labelsize='small', labelrotation=90)
plt.rcParams["figure.figsize"] = (12,6)

##### Exercise 4: Identify the highest paid apps with a good rating

In [ ]:
# YOUR CODE HERE
df_highest_paid_good_rating = df.sort_values(['Price', 'Rating'], ascending=False, ignore_index=True)
df_highest_paid_good_rating = df_highest_paid_good_rating[df_highest_paid_good_rating['Rating']>=4]

print("The top 10 highest paid apps with a good rating are: \n")
print(df_highest_paid_good_rating[['App', 'Price', 'Rating']].head(10))

##### Exercise 5: Are the top-rated apps genuine ? How about checking reviews count of top-rated apps ?

In [ ]:
# YOUR CODE HERE
df_top_rated = df.sort_values(['Rating', 'Reviews'], ascending=[False, True])

print("The top rated apps with review counts are: \n")
print(df_top_rated[['App', 'Rating', 'Reviews']].head(50))

##### Exercise 6: If the number of reviews of an app is very low, what could be the reason for its top-rating ?

In [ ]:
# YOUR CODE HERE
df_low_review = df.sort_values(['Reviews', 'Rating'], ascending=[True, False]).head(9000)
sns.scatterplot(x=df_low_review['Installs'],y=df_low_review['Rating'], hue=df['Reviews'])

##### Exercise 7: What is the 95% confidence interval for the rating of apps in the Google Play Store?

In [ ]:
# YOUR CODE HERE
import scipy.stats as st

print('mean = ', np.mean(df['Rating']))
print('se = ', st.sem(df['Rating'], nan_policy='omit'))

ci_t_with_95 = st.t.interval(confidence=0.95, df=df.shape[0]-1, loc=np.mean(df['Rating']), scale=st.sem(df['Rating'], nan_policy='omit'))

ci_norm_with_95 = st.norm.interval(confidence=0.95, loc=np.mean(df['Rating']), scale=st.sem(df['Rating'], nan_policy='omit'))

print("CI (using normal distribution) = ", ci_norm_with_95)

##### Exercise 8: Test if there is a statistically significant difference in the ratings between free and paid apps using a t-test

Steps:

* Set the null hypothesis and alternate hypothesis
* Separate the ratings of free and paid apps.
* Perform t-test: Use an independent samples t-test.
* Interpret results based on the p-value, decide whether to reject or fail to reject the null hypothesis.

In [ ]:
# YOUR CODE HERE
import scipy.stats as st
# H0 : there is no significant difference between free and paid apps when it comes to rating
# HA : there is a difference between free and paid apps on rating

df_free_ratings = df[df['Type'] == 'Free']['Rating'].dropna()
df_paid_ratings = df[df['Type'] == 'Paid']['Rating'].dropna()

print("Mean of ratings of free apps = ", round(df_free_ratings.mean(),4))
print("Mean of ratings of paid apps = ", round(df_paid_ratings.mean(), 4))

ttest = st.ttest_ind(a = df_free_ratings, b = df_paid_ratings, equal_var= False, nan_policy='omit')
print('t-stat= ', round(ttest.statistic,4))
print('p value= ', round(ttest.pvalue,6))
# p value = 4e-06 is less than 0.05, so H0 is rejected.
# That is - here is a significant difference between paid and free apps when it comes to rating

import statsmodels.api as sm
# Perform Two sample Z-test for Mean
two_sample_ztest = sm.stats.ztest(x1 = df_free_ratings,
                                  x2 = df_paid_ratings,
                                  value = 0,
                                  alternative = 'two-sided')    # Assume samples have equal variance
print("test-statistic (z-score): ", two_sample_ztest[0])
print("p-value: ", two_sample_ztest[1])